In [2]:
import glob
from google.colab import drive
drive.mount('/content/drive')

base_path = "/content/drive/My Drive/Poemes"

for path in glob.glob(base_path):
  print(path)

Mounted at /content/drive
/content/drive/My Drive/Poemes


In [4]:
#bibliothèque sacremose pour tokeniser
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 9.0 MB/s eta 0:00:00


In [5]:
#version de Flaubert encore avec le modèle NER existant basé sur CamemBERT : Jean-Baptiste/camembert-ner de Hugging Face

import os
import json
from tqdm import tqdm
from transformers import AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained("flaubert/flaubert_base_cased")
nlp_ner = pipeline("ner", model="Jean-Baptiste/camembert-ner")

base_path = "/content/drive/My Drive/Poemes/"
dic = {}

entites_par_auteur = {}
tokens_par_auteur = {}

for autor in tqdm(os.listdir(base_path), desc="Autors", unit="auteur"):
    autor_path = os.path.join(base_path, autor)

    if os.path.isdir(autor_path):
        entites_par_auteur[autor] = 0
        tokens_par_auteur[autor] = 0

        for poema in tqdm(os.listdir(autor_path), desc=f"Poèmes de {autor}", unit="poème", leave=False):
            if poema.endswith(".txt"):
                file_path = os.path.join(autor_path, poema)

                with open(file_path, "r", encoding="utf-8") as f:
                    conteudo = f.read()

                tokens = tokenizer.tokenize(conteudo)
                nb_tokens = len(tokens)

                try:
                    entidades = nlp_ner(conteudo)
                    nb_entites = len(entidades)

                    tokens_par_auteur[autor] += nb_tokens
                    entites_par_auteur[autor] += nb_entites

                    dic[poema] = {
                        "auteur": autor,
                        "nombre de tokens": nb_tokens,
                        "nombre d'entités nommées": nb_entites,
                        "entités nommées": [{"texte": ent["word"], "type": ent["entity"]} for ent in entidades],
                    }
                except Exception as e:
                    print(f"❗ Erreur lors du traitement de {poema} ({autor}) : {e}")

json_file_path = "/content/drive/My Drive/premier_resultat_flaubert.json"
with open(json_file_path, "w", encoding="utf-8") as json_file:
    json.dump(dic, json_file, ensure_ascii=False, indent=4)


print("\nNombre total de tokens par auteur :")
for auteur, nb_tokens in tokens_par_auteur.items():
    print(f"- {auteur}: {nb_tokens} tokens")

print("\nNombre total d'entités nommées par auteur :")
for auteur, nb_entites in entites_par_auteur.items():
    print(f"- {auteur}: {nb_entites} entités nommées")

total_tokens = sum(tokens_par_auteur.values())
total_entites = sum(entites_par_auteur.values())

print(f"\nNombre total de tokens dans tous les poèmes : {total_tokens}")
print(f"Nombre total d'entités nommées dans tous les poèmes : {total_entites}")

print(f"\n✅ Analyse terminée. Résultats enregistrés dans {json_file_path}.")


config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

Device set to use cpu
Autors: 100%|██████████| 10/10 [00:31<00:00,  3.11s/auteur]



Nombre total de tokens par auteur :
- Appolinaire: 19808 tokens
- Darbourville: 60937 tokens
- Desbordes: 35102 tokens
- Hugo: 75902 tokens
- Loiseau: 22432 tokens
- Noialles: 14641 tokens
- Rimbaud: 23008 tokens
- Sauvage: 24103 tokens
- Verlaine: 15719 tokens
- Vivien: 45851 tokens

Nombre total d'entités nommées par auteur :
- Appolinaire: 75 entités nommées
- Darbourville: 127 entités nommées
- Desbordes: 54 entités nommées
- Hugo: 94 entités nommées
- Loiseau: 87 entités nommées
- Noialles: 101 entités nommées
- Rimbaud: 91 entités nommées
- Sauvage: 90 entités nommées
- Verlaine: 125 entités nommées
- Vivien: 157 entités nommées

Nombre total de tokens dans tous les poèmes : 337503
Nombre total d'entités nommées dans tous les poèmes : 1001

✅ Analyse terminée. Résultats enregistrés dans /content/drive/My Drive/premier_resultat_flaubert.json.
